In [2]:
import polars as pl
from src.tank_ts_endpoint import fetch_tank_ts_data, GetTanksTimeDataReq

#pl.Config.set_tbl_rows(2000)

req = GetTanksTimeDataReq(source_key=['9884001', '98840FAC'])

df = await fetch_tank_ts_data(req)
if df is None:
    raise ValueError("df is None")
lf = df.lazy()
lf.collect()

ts,value,source_key,metric_nice_name,uom
datetime[μs],f64,str,str,str
2018-01-11 09:12:20.082822,299.173063,"""9884001""","""OilTank1Volume""","""bbl"""
2018-01-11 08:12:20.082822,299.066591,"""9884001""","""OilTank1Volume""","""bbl"""
2018-01-11 07:12:20.082822,298.999721,"""9884001""","""OilTank1Volume""","""bbl"""
2018-01-11 06:12:20.082822,299.004504,"""9884001""","""OilTank1Volume""","""bbl"""
2018-01-11 05:12:20.082822,299.105416,"""9884001""","""OilTank1Volume""","""bbl"""
…,…,…,…,…
2024-07-26 22:42:10.494432,87.474737,"""9884001""","""OilTank1Level""","""in"""
2024-07-26 21:42:10.494432,85.827355,"""9884001""","""OilTank1Level""","""in"""
2024-07-26 20:42:10.494432,84.148518,"""9884001""","""OilTank1Level""","""in"""


In [3]:
tank_metrics = ["Level", "Volume", "InchesUntilAlarm", "InchesToESD"]
tank_types = ["Water", "Oil"]

tank_metrics_str= "|".join(tank_metrics)
tank_types_str = "|".join(tank_types)

pattern = f'^(?<is_esd>ESD-)?(?<tank_type>{tank_types_str})Tank(?<tank_number>[0-9]*)(?<tank_metric>{tank_metrics_str})'
lf = lf.with_columns(separated_metrics = pl.col("metric_nice_name").str.extract_groups(pattern))
lf = lf.unnest("separated_metrics")

lf = lf.drop("is_esd", "tank_number")

filtered_lf = lf.filter(~pl.col("metric_nice_name").str.ends_with("ID"))

ts_to_sec = filtered_lf.with_columns(
    pl.col("ts").dt.strftime("%Y-%m-%d %H:%M:%S").str.strptime(pl.Datetime).alias("ts")
)


aggregated_lf = ts_to_sec.group_by(["tank_metric", "tank_type","uom"]).agg([
    pl.col("ts").alias("timestamps"),
    pl.col("value").alias("values")
])

aggregated_lf = aggregated_lf.sort("tank_metric", "tank_type")

aggregated_lf.collect()

tank_metric,tank_type,uom,timestamps,values
str,str,str,list[datetime[μs]],list[f64]
"""InchesUntilAlarm""","""Oil""","""in""","[2023-12-21 06:22:10, 2023-09-11 23:03:57, 2024-08-22 10:59:43]","[6.0, 130.227169, 102.478181]"
"""InchesUntilAlarm""","""Water""","""in""","[2024-07-27 02:42:26, 2024-05-22 01:16:33, 2024-08-16 04:45:27]","[1.0, 239.561629, 257.847738]"
"""Level""","""Oil""","""in""","[2024-07-28 17:42:10, 2024-07-28 16:42:10, … 2024-07-26 18:42:10]","[79.02198, 78.915508, … 80.737793]"
"""Volume""","""Oil""","""bbl""","[2018-01-11 09:12:20, 2018-01-11 08:12:20, … 2018-01-09 10:12:20]","[299.173063, 299.066591, … 300.888877]"


In [ ]:
tank_metrics = ["Level", "Volume", "InchesUntilAlarm", "InchesToESD"]
tank_types = ["Water", "Oil"]

tank_metrics_str= "|".join(tank_metrics)
tank_types_str = "|".join(tank_types)

pattern = f'^(?<is_esd>ESD-)?(?<tank_type>{tank_types_str})Tank(?<tank_number>[0-9]*)(?<tank_metric>{tank_metrics_str})'
lf = lf.with_columns(separated_metrics = pl.col("metric_nice_name").str.extract_groups(pattern))
lf = lf.unnest("separated_metrics")
lf.collect()

In [ ]:
lf = lf.drop("is_esd", "tank_number")
lf.collect()

In [ ]:
filtered_lf = lf.filter(~pl.col("metric_nice_name").str.ends_with("ID"))
filtered_lf.collect()

In [ ]:
ts_to_sec = filtered_lf.with_columns(
    pl.col("ts").dt.strftime("%Y-%m-%d %H:%M:%S").str.strptime(pl.Datetime).alias("ts")
)

ts_to_sec.collect()

In [ ]:
aggregated_lf = ts_to_sec.group_by(["tank_metric", "tank_type","uom"]).agg([
    pl.col("ts").alias("timestamps"),
    pl.col("value").alias("values")
])

aggregated_lf = aggregated_lf.sort("tank_metric")

aggregated_lf.collect()